<a href="https://colab.research.google.com/github/ScholliYT/AdventOfCode/blob/main/AdventOfCode2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [175]:
import numpy as np
from collections import Counter
from functools import reduce
from functools import partial
import collections
from dataclasses import dataclass
from tqdm import tqdm
import operator
import math

# Day 1

In [ ]:
with open('day1_input.txt') as file:
    lines = file.readlines()

len(lines)

FileNotFoundError: ignored

In [ ]:
numbers = list(map(int, lines))

def get_increase_cnt(arr):
  total = reduce(lambda t, nxt: (nxt, t[1] +1 if nxt > t[0] else t[1]), arr[1:], (arr[0], 0))
  return total[1]

get_increase_cnt(numbers)

In [ ]:
def append_to_and_ret(deque, val):
  deque.append(val)
  return deque


def sliding_window_sum(arr):
  window = collections.deque(maxlen=3)
  window.extend(arr[:3])
  total = reduce(lambda t, nxt: (append_to_and_ret(t[0], nxt), t[1] + [sum(t[0])]), arr[3:], (window, [sum(window)]))
  return total[1]

sums = sliding_window_sum(numbers)
get_increase_cnt(sums)

# Day 2

In [ ]:
with open('day2_input.txt') as file:
    lines = file.readlines()

len(lines)

In [ ]:
def parse(line):
  cmd, amount = line.split(' ')
  amount = int(amount)
  if cmd == 'forward':
    return (amount, 0)
  elif cmd == 'up':
    return (0, -amount)
  elif cmd == 'down':
    return (0, amount)


res = [sum(x) for x in zip(*list(map(parse, lines)))]
res[0] * res[1]

In [ ]:
def parse(line, coords):
  cmd, amount = line.split(' ')
  amount = int(amount)
  if cmd == 'forward':
    return (coords[0] + amount, coords[1] + coords[2]*amount, coords[2])
  elif cmd == 'up':
    return (coords[0], coords[1], coords[2] -amount)
  elif cmd == 'down':
    return (coords[0], coords[1], coords[2] +amount)


res = reduce(lambda coords, line: parse(line, coords), lines, (0,0,0))
res[0] * res[1]

# Day 3

In [ ]:
with open('day3_input.txt') as file:
    lines = file.readlines()

len(lines)

In [ ]:
bits = list(map(list, lines))
arr = np.array(bits)[:, :-1]

In [ ]:
def most_common_per_position(arr):
  common = map(lambda most_common: '1' if most_common['0']<=most_common['1'] else '0', [dict(Counter(row).most_common()) for row in arr.transpose()])
  return ''.join(common)

gamma_rate_s = most_common_per_position(arr)
epsilon_rate_s = ''.join(['1' if i == '0' else '0' for i in gamma_rate_s])

int(gamma_rate_s, 2) * int(epsilon_rate_s, 2)

In [ ]:
# arr = np.array([
#                 ['0', '1', '0', '0'],
#                 ['0', '1', '1', '0'],
#                 ['1', '1', '1', '0'],
#                 ['1', '0', '1', '1'],
#                 ['1', '0', '1', '0'],
#                 ['1', '0', '0', '0']])


def most_common_per_position_default_1(arr):
  dicts = [dict(Counter(row).most_common()) for row in arr.transpose()]
  common = map(lambda most_common: list(most_common.keys())[0] if len(most_common) == 1 else ('1' if most_common['0']<=most_common['1'] else '0'), dicts)
  return ''.join(common)

oxygen_arr = arr.copy()
pos = 0
while oxygen_arr.shape[0] > 1:
  common = most_common_per_position_default_1(oxygen_arr)
  oxygen_arr = np.array([row for row in oxygen_arr if row[pos] == common[pos]])
  pos += 1

oxygen_generator_rating_s = ''.join(oxygen_arr[0])



def least_common_per_position_default_0(arr):
  dicts = [dict(Counter(row).most_common()) for row in arr.transpose()]
  common = map(lambda most_common: list(most_common.keys())[0] if len(most_common) == 1 else ('0' if most_common['0']<=most_common['1'] else '1'), dicts)
  return ''.join(common)

co2_arr = arr.copy()
pos = 0
while co2_arr.shape[0] > 1:
  uncommon = least_common_per_position_default_0(co2_arr)
  co2_arr = np.array([row for row in co2_arr if row[pos] == uncommon[pos]])
  pos += 1

CO2_scrubber_rating_s = ''.join(co2_arr[0])




display(int(oxygen_generator_rating_s, 2) * int(CO2_scrubber_rating_s, 2))

# Day 4

In [ ]:
with open('day4_input.txt') as file:
    lines = file.readlines()

len(lines)

draw_numbers = list(map(int, lines[0].split(',')))

board_defs = lines[2:]
cut_idxs = [idx for idx in range(len(board_defs)) if board_defs[idx]=='\n']
cut_ranges = [-1] + cut_idxs + [len(board_defs)+1]

boards = []
for start, end in zip(cut_ranges, cut_ranges[1:]):
  boards.append([list(map(int, filter(None, row.split(' ')))) for row in board_defs[start+1:end]])
boards = np.asarray(boards)
display(boards.shape)

(100, 5, 5)

In [ ]:
def mark(boards, old_marked, value):
  idx_to_mark = np.where(boards.flatten() == value)[0]
  marked = old_marked.flatten()
  marked[idx_to_mark] = True
  return marked.reshape(old_marked.shape)

# input a single board of shape i.e. 5x5
def is_winning_state(board):
  win_by_row = np.any(np.all(board, axis=1))
  win_by_col = np.any(np.all(board, axis=0))
  return win_by_row or win_by_col

In [ ]:
def get_first_winning_boards(boards, draw_numbers):
  marking = np.full(boards.shape, False, dtype=bool)
  for idx,number in enumerate(draw_numbers):
    marking = mark(boards, marking, number)

    winning_states = np.array([is_winning_state(board) for board in marking])
    if np.any(winning_states):
      return number, np.where(winning_states==True)[0], marking

final_number, winning_boards, marking = get_first_winning_boards(boards, draw_numbers)
winning_board_idx = winning_boards[0]
winning_board = boards[winning_board_idx]

display(final_number, winning_boards, winning_board_idx, marking[winning_board_idx], winning_board)

false_idxs = np.where(marking[winning_board_idx].flatten()==False)[0]
np.sum(winning_board.flatten()[false_idxs]) * final_number

78

array([45])

45

array([[ True, False,  True, False,  True],
       [ True, False, False, False,  True],
       [ True,  True, False,  True, False],
       [ True, False, False, False, False],
       [ True, False, False, False,  True]])

array([[81, 23,  6, 14, 26],
       [86, 42, 82, 95, 85],
       [77, 52, 38,  2, 33],
       [69, 98, 54, 37,  4],
       [78, 39,  3, 75, 80]])

49686

In [ ]:
def get_last_winning_boards(boards, draw_numbers):
  marking = np.full(boards.shape, False, dtype=bool)
  for idx,number in enumerate(draw_numbers):
    winning_states = np.array([is_winning_state(board) for board in marking])
    not_yet_winning_boards = np.where(winning_states==False)[0]
    
    marking = mark(boards, marking, number)

    winning_states = np.array([is_winning_state(board) for board in marking])
    if np.all(winning_states):
      return number, not_yet_winning_boards, marking

final_number, winning_boards, marking = get_last_winning_boards(boards, draw_numbers)
winning_board_idx = winning_boards[0]
winning_board = boards[winning_board_idx]

display(final_number, winning_boards, winning_board_idx, marking[winning_board_idx], winning_board)

false_idxs = np.where(marking[winning_board_idx].flatten()==False)[0]
np.sum(winning_board.flatten()[false_idxs]) * final_number

89

array([43])

43

array([[False,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True, False,  True,  True],
       [ True,  True,  True,  True, False],
       [ True, False,  True, False,  True]])

array([[87, 90, 16, 56, 67],
       [41, 75, 89,  1, 80],
       [22, 62,  5, 45, 69],
       [28, 36, 19, 96, 71],
       [26, 63, 88, 76, 31]])

26878

# Day 5

In [ ]:
with open('day5_input.txt') as file:
    input_lines = file.readlines()
len(input_lines)

500

In [ ]:
lines = np.array(list(map(lambda line: list(map(lambda point: list(map(int, point.split(','))), line)), map(lambda s: s.split(' -> '), input_lines))))
lines.shape

(500, 2, 2)

In [ ]:
# get the min and max coord (in all axis)
min_coord = lines.flatten().min()
max_coord = lines.flatten().max()

assert min_coord > 0

field = np.zeros((max_coord+1, max_coord+1))

In [ ]:
field_1 = field.copy()

for line in lines:
  x_start, y_start = line[0]
  x_end, y_end = line[1]
  if x_start == x_end or y_start == y_end:

    print(line)
    
    if x_start > x_end:
      x_start, x_end = (x_end, x_start)
      y_start, y_end = (y_end, y_start)
    elif y_start > y_end:
      x_start, x_end = (x_end, x_start)
      y_start, y_end = (y_end, y_start)

    print(f"increasing area ({x_start}:{x_end+1}) ({y_start}:{y_end+1})")
    field_1[y_start:(y_end+1), x_start:(x_end+1)] = field_1[y_start:(y_end+1), x_start:(x_end+1)] + 1

display(np.where(field_1.flatten()>=2)[0].shape[0])

[[735  73]
 [316  73]]
increasing area (316:736) (73:74)
[[981 566]
 [981  11]]
increasing area (981:982) (11:567)
[[631 588]
 [631 910]]
increasing area (631:632) (588:911)
[[502 248]
 [774 248]]
increasing area (502:775) (248:249)
[[444 815]
 [444 554]]
increasing area (444:445) (554:816)
[[734  97]
 [734 210]]
increasing area (734:735) (97:211)
[[871 692]
 [368 692]]
increasing area (368:872) (692:693)
[[842 261]
 [606 261]]
increasing area (606:843) (261:262)
[[332  43]
 [206  43]]
increasing area (206:333) (43:44)
[[594 782]
 [127 782]]
increasing area (127:595) (782:783)
[[827 741]
 [827 168]]
increasing area (827:828) (168:742)
[[299 598]
 [299 588]]
increasing area (299:300) (588:599)
[[478  76]
 [607  76]]
increasing area (478:608) (76:77)
[[566 447]
 [857 447]]
increasing area (566:858) (447:448)
[[354 721]
 [538 721]]
increasing area (354:539) (721:722)
[[124  17]
 [124 339]]
increasing area (124:125) (17:340)
[[105 595]
 [843 595]]
increasing area (105:844) (595:596)
[[683 

6005

In [ ]:
field_2 = field_1.copy()

for line in lines:
  x_start, y_start = line[0]
  x_end, y_end = line[1]
  if x_start != x_end and y_start != y_end: # only diagonal (45 degree)
    
    #print(line)
    x_range = range(x_start, x_end+1)  
    if x_start > x_end:
      x_range = range(x_start, x_end-1, -1)

    y_range = range(y_start, y_end+1)
    if y_start > y_end:
      y_range = range(y_start, y_end-1, -1)

    assert len(x_range) == len(y_range), f"{len(x_range)} != {len(y_range)} " + str(line)

    for x,y in zip(x_range, y_range):
      #print("Marking point", x, y)
      field_2[y,x] = field_2[y,x] + 1

display(np.where(field_2.flatten()>=2)[0].shape[0])

23864

# Day 10

In [116]:
with open('day10_input.txt') as file:
    input_lines = file.readlines()
len(input_lines)

106

In [167]:
line = "(<)"

def is_corrupt(line):
  opening = ["(", "[", "{", "<"]
  closing = [")", "]", "}", ">"]
  open = []

  last_open = None
  for idx,c in enumerate(line):
    if c in opening:
      open.append(c)
    elif c in closing:
      cidx = closing.index(c)
      if open[-1] == opening[cidx]:
        open.pop()
      else:
        oidx = opening.index(open[-1])
        #print(f"Syntax Error at index {idx}. Found {c} but expected {closing[oidx]}.")
        return True, cidx
    else:
      raise Exception(f"Unknown character: {c}")
  return False, None


scores = [3, 57, 1197, 25137]
sum(map(lambda t: scores[t[1]], filter(operator.itemgetter(0), map(lambda line: is_corrupt(line[:-1]), input_lines))))

436497

In [226]:
non_corrupt_lines = list(filter(lambda line: not is_corrupt(line[:-1])[0], input_lines))

# swaps order of iterable and initial_value
def my_reduce(function, initial_value, iterable):
  return reduce(function, iterable, initial_value)

# f_reduce is just like my_reduce
# it leaves the first pameter (namely function) as is and swaps the order of the following ones
flip = lambda f: lambda *a: f(a[0], *reversed(a[1:]))
f_reduce = flip(reduce)

def get_completion(line):
  opening = ["(", "[", "{", "<"]
  closing = [")", "]", "}", ">"]
  open = []

  last_open = None
  for idx,c in enumerate(line):
    if c in opening:
      open.append(c)
    elif c in closing:
      cidx = closing.index(c)
      if open[-1] == opening[cidx]:
        open.pop()
      else:
        raise Exception(f"Syntax Error at index {idx}. Found {c} but expected {closing[oidx]}.")
    else:
      raise Exception(f"Unknown character: {c}")

  completion = list(map(lambda c: opening.index(c), reversed(open)))
  return completion


scores = [1,2,3,4]
completions = map(lambda line: get_completion(line[:-1]), non_corrupt_lines)
per_line_score = list(map(partial(f_reduce, lambda line_score, completion_char_idx: line_score*5 + scores[completion_char_idx], 0), completions))
sorted(per_line_score)[len(per_line_score)//2]

2377613374

# Day 11

In [18]:
with open('day11_input.txt') as file:
    input_lines = file.readlines()
len(input_lines)

10

In [48]:
grid = np.array([[int(c) for c in row[:-1]] for row in input_lines])
grid, grid.shape

(array([[2, 3, 4, 4, 6, 7, 1, 2, 1, 2],
        [6, 6, 1, 1, 7, 4, 2, 6, 8, 1],
        [5, 5, 7, 5, 5, 7, 5, 5, 7, 3],
        [3, 1, 6, 7, 8, 4, 8, 5, 3, 6],
        [1, 3, 5, 3, 8, 2, 7, 3, 1, 1],
        [4, 4, 1, 6, 4, 6, 3, 2, 6, 6],
        [2, 6, 2, 4, 7, 6, 1, 6, 1, 5],
        [1, 7, 8, 6, 5, 6, 1, 2, 6, 3],
        [3, 6, 2, 2, 6, 4, 3, 2, 1, 5],
        [4, 1, 4, 3, 2, 8, 4, 6, 5, 3]]), (10, 10))

In [101]:
x = 9
y = 9
ngrid = grid.copy()
ngrid[max(y-1, 0):min(y+2, 10), max(x-1, 0):min(x+2, 10)] = ngrid[max(y-1, 0):min(y+2, 10), max(x-1, 0):min(x+2, 10)] + 1
ngrid

array([[2, 3, 4, 4, 6, 7, 1, 2, 1, 2],
       [6, 6, 1, 1, 7, 4, 2, 6, 8, 1],
       [5, 5, 7, 5, 5, 7, 5, 5, 7, 3],
       [3, 1, 6, 7, 8, 4, 8, 5, 3, 6],
       [1, 3, 5, 3, 8, 2, 7, 3, 1, 1],
       [4, 4, 1, 6, 4, 6, 3, 2, 6, 6],
       [2, 6, 2, 4, 7, 6, 1, 6, 1, 5],
       [1, 7, 8, 6, 5, 6, 1, 2, 6, 3],
       [3, 6, 2, 2, 6, 4, 3, 2, 2, 6],
       [4, 1, 4, 3, 2, 8, 4, 6, 6, 4]])

In [106]:
display(grid + 1)
display(grid + 1 +1)
display(np.where((grid +1 +1).flatten() >= 10))

array([[3, 4, 5, 5, 7, 8, 2, 3, 2, 3],
       [7, 7, 2, 2, 8, 5, 3, 7, 9, 2],
       [6, 6, 8, 6, 6, 8, 6, 6, 8, 4],
       [4, 2, 7, 8, 9, 5, 9, 6, 4, 7],
       [2, 4, 6, 4, 9, 3, 8, 4, 2, 2],
       [5, 5, 2, 7, 5, 7, 4, 3, 7, 7],
       [3, 7, 3, 5, 8, 7, 2, 7, 2, 6],
       [2, 8, 9, 7, 6, 7, 2, 3, 7, 4],
       [4, 7, 3, 3, 7, 5, 4, 3, 2, 6],
       [5, 2, 5, 4, 3, 9, 5, 7, 6, 4]])

array([[ 4,  5,  6,  6,  8,  9,  3,  4,  3,  4],
       [ 8,  8,  3,  3,  9,  6,  4,  8, 10,  3],
       [ 7,  7,  9,  7,  7,  9,  7,  7,  9,  5],
       [ 5,  3,  8,  9, 10,  6, 10,  7,  5,  8],
       [ 3,  5,  7,  5, 10,  4,  9,  5,  3,  3],
       [ 6,  6,  3,  8,  6,  8,  5,  4,  8,  8],
       [ 4,  8,  4,  6,  9,  8,  3,  8,  3,  7],
       [ 3,  9, 10,  8,  7,  8,  3,  4,  8,  5],
       [ 5,  8,  4,  4,  8,  6,  5,  4,  3,  7],
       [ 6,  3,  6,  5,  4, 10,  6,  8,  7,  5]])

(array([18, 34, 36, 44, 72, 95]),)

In [114]:
next_grid = grid.copy()

all_flash_step = -1

for step in range(1000):
  #display("Step", step)
  next_grid = next_grid + 1 # initial increase

  #display(next_grid)

  flashed = np.zeros(grid.shape)

  flashing_idxs = np.where(next_grid.flatten() >= 10)[0]
  next_grid_after_flash = next_grid.copy()

  #display("flashing_idxs", flashing_idxs)

  for flashing_idx in flashing_idxs:
    if flashed.flatten()[flashing_idx] == 0:
      x = flashing_idx % 10
      y = flashing_idx // 10
      next_grid_after_flash[max(y-1, 0):min(y+2, 10), max(x-1, 0):min(x+2, 10)] = next_grid_after_flash[max(y-1, 0):min(y+2, 10), max(x-1, 0):min(x+2, 10)] + 1
      flashed[y,x] = 1
      flash_counter += 1

  #display(next_grid_after_flash)

  while not (next_grid == next_grid_after_flash).all():
    next_grid = next_grid_after_flash.copy()
    flashing_idxs = np.where(next_grid.flatten() >= 10)[0]

    #display("flashing_idxs", flashing_idxs)
    for flashing_idx in flashing_idxs:
      if flashed.flatten()[flashing_idx] == 0:

        x = flashing_idx % 10
        y = flashing_idx // 10
        next_grid_after_flash[max(y-1, 0):min(y+2, 10), max(x-1, 0):min(x+2, 10)] = next_grid_after_flash[max(y-1, 0):min(y+2, 10), max(x-1, 0):min(x+2, 10)] + 1
        flashed[y,x] = 1
        flash_counter += 1

      #display(next_grid_after_flash)

  #display(next_grid_after_flash)

  tmp = next_grid.flatten()
  tmp[np.where(flashed.flatten() == 1)[0]] = 0
  next_grid = tmp.reshape(next_grid.shape)

  if len(np.where(flashed.flatten() == 1)[0]) == len(grid.flatten()):
    all_flash_step = step +1
    break

  #display(next_grid_after_flash)

next_grid, all_flash_step

(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 237)

# Day 12

In [2]:
with open('day12_input.txt') as file:
    input_lines = file.readlines()
len(input_lines)

24

In [3]:
paths = list(map(lambda path: path[:-1].split('-'), input_lines))
paths

[['hl', 'WP'],
 ['vl', 'fo'],
 ['vl', 'WW'],
 ['WP', 'start'],
 ['vl', 'QW'],
 ['fo', 'wy'],
 ['WW', 'dz'],
 ['dz', 'hl'],
 ['fo', 'end'],
 ['VH', 'fo'],
 ['ps', 'vl'],
 ['FN', 'dz'],
 ['WP', 'ps'],
 ['ps', 'start'],
 ['WW', 'hl'],
 ['end', 'QW'],
 ['start', 'vl'],
 ['WP', 'fo'],
 ['end', 'FN'],
 ['hl', 'QW'],
 ['WP', 'dz'],
 ['QW', 'fo'],
 ['QW', 'dz'],
 ['ps', 'dz']]

In [4]:
nodes = np.unique(np.array(paths).flatten())
len(nodes), nodes

(13, array(['FN', 'QW', 'VH', 'WP', 'WW', 'dz', 'end', 'fo', 'hl', 'ps',
        'start', 'vl', 'wy'], dtype='<U5'))

In [5]:
adj_list = {node: set([p[1] for p in paths if p[0]==node] + [p[0] for p in paths if p[1]==node]) for node in nodes}
adj_list

{'FN': {'dz', 'end'},
 'QW': {'dz', 'end', 'fo', 'hl', 'vl'},
 'VH': {'fo'},
 'WP': {'dz', 'fo', 'hl', 'ps', 'start'},
 'WW': {'dz', 'hl', 'vl'},
 'dz': {'FN', 'QW', 'WP', 'WW', 'hl', 'ps'},
 'end': {'FN', 'QW', 'fo'},
 'fo': {'QW', 'VH', 'WP', 'end', 'vl', 'wy'},
 'hl': {'QW', 'WP', 'WW', 'dz'},
 'ps': {'WP', 'dz', 'start', 'vl'},
 'start': {'WP', 'ps', 'vl'},
 'vl': {'QW', 'WW', 'fo', 'ps', 'start'},
 'wy': {'fo'}}

In [10]:
# (path: ["start","WP"], visited: {'start', 'WP', 'ps'}, next: 'ps')

paths_to_end = []

@dataclass
class Path:
  path: [str]
  visited: set
  next: str

start_path = Path([], set([]), "start")
queue = [start_path]

while len(queue) > 0:
  cur_path = queue.pop(0)
  #display("cur_path", cur_path)

  if cur_path.next == "end":
    paths_to_end.append(Path(cur_path.path + [cur_path.next], cur_path.visited.union({cur_path.next}), None))
  else:
    neighbours = adj_list[cur_path.next]
    #display("neighbours", neighbours)

    next_paths = [Path(cur_path.path + [cur_path.next], cur_path.visited.union({cur_path.next}), n) for n in neighbours]
    next_paths = filter(lambda p: p.next.isupper() or p.next not in p.visited, next_paths)
    queue.extend(next_paths)
    #display("Queue", queue)

len(paths_to_end), paths_to_end

(3410,
 [Path(path=['start', 'vl', 'fo', 'end'], visited={'end', 'vl', 'start', 'fo'}, next=None),
  Path(path=['start', 'vl', 'QW', 'end'], visited={'end', 'vl', 'start', 'QW'}, next=None),
  Path(path=['start', 'WP', 'fo', 'end'], visited={'end', 'fo', 'start', 'WP'}, next=None),
  Path(path=['start', 'vl', 'fo', 'QW', 'end'], visited={'end', 'vl', 'fo', 'start', 'QW'}, next=None),
  Path(path=['start', 'vl', 'QW', 'fo', 'end'], visited={'end', 'vl', 'fo', 'start', 'QW'}, next=None),
  Path(path=['start', 'ps', 'dz', 'FN', 'end'], visited={'end', 'dz', 'ps', 'FN', 'start'}, next=None),
  Path(path=['start', 'ps', 'dz', 'QW', 'end'], visited={'end', 'dz', 'ps', 'start', 'QW'}, next=None),
  Path(path=['start', 'ps', 'vl', 'fo', 'end'], visited={'end', 'vl', 'ps', 'fo', 'start'}, next=None),
  Path(path=['start', 'ps', 'vl', 'QW', 'end'], visited={'end', 'vl', 'ps', 'start', 'QW'}, next=None),
  Path(path=['start', 'ps', 'WP', 'fo', 'end'], visited={'end', 'ps', 'fo', 'WP', 'start'}, n

In [17]:
# (path: ["start","WP"], visited: {'start', 'WP', 'ps'}, next: 'ps')

paths_to_end = []

@dataclass
class Path:
  path: [str]
  visited: set
  double_visit: bool
  next: str

start_path = Path([], set([]), False, "start")
queue = [start_path]

while len(queue) > 0:
  cur_path = queue.pop(0)
  #display("cur_path", cur_path)

  if cur_path.next == "end":
    paths_to_end.append(Path(cur_path.path + [cur_path.next], cur_path.visited.union({cur_path.next}), cur_path.double_visit, None))
  else:
    neighbours = adj_list[cur_path.next]
    #display("neighbours", neighbours)

    next_paths = [Path(cur_path.path + [cur_path.next], cur_path.visited.union({cur_path.next}), cur_path.double_visit, n) for n in neighbours]
    next_paths = filter(lambda p: p.next.isupper() or p.next not in p.visited, next_paths)

    double_visist_paths = [Path(cur_path.path + [cur_path.next], cur_path.visited.union({cur_path.next}), True, n) for n in neighbours if n.islower() and not cur_path.double_visit and n in cur_path.visited and n not in ["start", "end"]]
    queue.extend(list(next_paths) + double_visist_paths)
    #display("Queue", queue)

len(paths_to_end)
#len(paths_to_end), paths_to_end

98796